In [1]:
import os

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

from model import simple_mlp as mlp_float
from model_fixed import simple_mlp as mlp_fixed

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
dataset = input_data.read_data_sets('./data', one_hot = False) #read MNIST Dataset


######### float graph build ##########
#model define
model = mlp_float(
    n_layer = 3,
    hidden_dim = 512
)


#placeholder for batch data
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None,])

#build network graph
logit = model(x)
    
#define loss and optimizers
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = logit) 
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.to_int32(tf.argmax(logit, axis=-1)), y)))

saver = tf.train.Saver()
###### end graph build ######


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#pretrained model load test

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #load pretrained model
    saver.restore(sess, './results/model')

    ### test trained model ###
    test_loss, test_acc = sess.run(
            [loss, acc],
            feed_dict = {x: dataset.test.images, y: dataset.test.labels}
            )

    print('test loss : %.4f, acc : %.4f'%(test_loss, test_acc))

INFO:tensorflow:Restoring parameters from ./results/model
test loss : 0.0782, acc : 0.9832


In [30]:
tf.reset_default_graph()

In [31]:
_N_W = 2
_N_B = 4
_N_A = 4

######### fixed graph build ##########
#model define
model = mlp_fixed(
    n_layer = 3,
    hidden_dim = 512
)


#placeholder for batch data
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None,])

#build network graph
#logit = model(x)
logit = model(x, n_w = _N_W, n_b = _N_B, n_a = _N_A)
    
#define loss and optimizers
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = logit) 
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.to_int32(tf.argmax(logit, axis=-1)), y)))

saver = tf.train.Saver()
###### end graph build ######

In [32]:
#pretrained model direct quantization test

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #load pretrained model
    saver.restore(sess, './results/model')

    ### test trained model ###
    test_loss, test_acc = sess.run(
            [loss, acc],
            feed_dict = {x: dataset.test.images, y: dataset.test.labels}
            )

    print('test loss : %.4f, acc : %.4f'%(test_loss, test_acc))
    
    #for param check
    all_p = sess.run(tf.global_variables())
    w_list, b_list, a_list = sess.run(
            [model.fixed_w, model.fixed_b, model.fixed_a],
            feed_dict = {x: dataset.test.images}
            )
    w_clip, b_clip = sess.run([model.clip_list_w, model.clip_list_b])

INFO:tensorflow:Restoring parameters from ./results/model
test loss : 0.1095, acc : 0.9648


In [33]:
np.set_printoptions(precision = 5, suppress = True)
# check float variable 
for p, pp in zip(all_p, tf.global_variables()):
        if '1' in pp.name:
            print(pp.name)
            print(p[0:5])


w_1:0
[[-0.03323 -0.06025 -0.0491  ...  0.06586 -0.03667  0.04038]
 [-0.09036  0.04966 -0.04223 ...  0.08582 -0.08799 -0.00064]
 [ 0.07698  0.06667 -0.03831 ...  0.05336 -0.07198 -0.02152]
 [-0.06299 -0.02648 -0.00322 ... -0.08102 -0.05288 -0.0389 ]
 [ 0.02165  0.06696  0.00328 ... -0.06349 -0.04011  0.03164]]
b_1:0
[ 0.00732  0.00679 -0.00058  0.01099  0.02571]


In [34]:
#fixed weights in layer 1
w1 = w_list[1]
w1_clip = w_clip[1]
delta_w = (w1_clip - (-w1_clip)) / (2**_N_W - 2)
print('clip value : ', w1_clip)
print('delta : ', delta_w)

clip value :  0.05623006
delta :  0.05623006075620651


In [35]:
print('fake quant weights : ', w1[0:5])

fake quant weights :  [[-0.05623 -0.05623 -0.05623 ...  0.05623 -0.05623  0.05623]
 [-0.05623  0.05623 -0.05623 ...  0.05623 -0.05623  0.     ]
 [ 0.05623  0.05623 -0.05623 ...  0.05623 -0.05623  0.     ]
 [-0.05623  0.       0.      ... -0.05623 -0.05623 -0.05623]
 [ 0.       0.05623  0.      ... -0.05623 -0.05623  0.05623]]


In [36]:
print('fixed weights : ', w1[0:5] / delta_w)

fixed weights :  [[-1. -1. -1. ...  1. -1.  1.]
 [-1.  1. -1. ...  1. -1.  0.]
 [ 1.  1. -1. ...  1. -1.  0.]
 [-1.  0.  0. ... -1. -1. -1.]
 [ 0.  1.  0. ... -1. -1.  1.]]


In [66]:
tf.reset_default_graph()

In [67]:
#### tf create fixed-point graph ####

######### float graph build ##########
#model define
model = mlp_float(
    n_layer = 3,
    hidden_dim = 512
)


#placeholder for batch data
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None,])

#build network graph
logit = model(x)
    
#define loss and optimizers
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = logit) 
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.to_int32(tf.argmax(logit, axis=-1)), y)))

saver = tf.train.Saver()
###### end (base) graph build ######

print('vars in default graphs')
for p in tf.global_variables():
    print(p.name)
       

vars in default graphs
w_0:0
b_0:0
w_1:0
b_1:0
w_2:0
b_2:0
w_3:0
b_3:0


In [68]:
_N_W = 8
_N_A = 4
#create quantization nodes
g = tf.get_default_graph()

#tf.contrib.quantize.create_eval_graph(input_graph = g)
    
tf.contrib.quantize.experimental_create_eval_graph(
        input_graph = g,
        weight_bits = _N_W,
        activation_bits = _N_A
        )
    
print('vars in fixed-point graphs')
for p in tf.global_variables():
    print(p.name)

vars in fixed-point graphs
w_0:0
b_0:0
w_1:0
b_1:0
w_2:0
b_2:0
w_3:0
b_3:0
xw_plus_b/weights_quant/min:0
xw_plus_b/weights_quant/max:0
xw_plus_b/act_quant/min:0
xw_plus_b/act_quant/max:0
xw_plus_b_1/weights_quant/min:0
xw_plus_b_1/weights_quant/max:0
xw_plus_b_1/act_quant/min:0
xw_plus_b_1/act_quant/max:0
xw_plus_b_2/weights_quant/min:0
xw_plus_b_2/weights_quant/max:0
xw_plus_b_2/act_quant/min:0
xw_plus_b_2/act_quant/max:0
xw_plus_b_3/weights_quant/min:0
xw_plus_b_3/weights_quant/max:0
xw_plus_b_3/act_quant/min:0
xw_plus_b_3/act_quant/max:0


In [69]:
#pretrained model direct quantization test

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #load pretrained model
    saver.restore(sess, './results/model')

    ### test trained model ###
    test_loss, test_acc = sess.run(
            [loss, acc],
            feed_dict = {x: dataset.test.images, y: dataset.test.labels}
            )

    print('test loss : %.4f, acc : %.4f'%(test_loss, test_acc))
    for p in tf.global_variables():
        if 'min' in p.name or 'max' in p.name:
            print(p.name, ' : ', sess.run(p))
            

INFO:tensorflow:Restoring parameters from ./results/model
test loss : 0.9473, acc : 0.5194
xw_plus_b/weights_quant/min:0  :  -6.0
xw_plus_b/weights_quant/max:0  :  6.0
xw_plus_b/act_quant/min:0  :  0.0
xw_plus_b/act_quant/max:0  :  6.0
xw_plus_b_1/weights_quant/min:0  :  -6.0
xw_plus_b_1/weights_quant/max:0  :  6.0
xw_plus_b_1/act_quant/min:0  :  0.0
xw_plus_b_1/act_quant/max:0  :  6.0
xw_plus_b_2/weights_quant/min:0  :  -6.0
xw_plus_b_2/weights_quant/max:0  :  6.0
xw_plus_b_2/act_quant/min:0  :  0.0
xw_plus_b_2/act_quant/max:0  :  6.0
xw_plus_b_3/weights_quant/min:0  :  -6.0
xw_plus_b_3/weights_quant/max:0  :  6.0
xw_plus_b_3/act_quant/min:0  :  0.0
xw_plus_b_3/act_quant/max:0  :  6.0


In [70]:
## get fixed-point params (by node name)

weights_quant_nodes = [n.name for n in tf.get_default_graph().as_graph_def().node if 'weights_quant' in n.name]
for node in weights_quant_nodes:
    print(node)

w1_fixed = g.get_tensor_by_name('xw_plus_b_1/weights_quant/FakeQuantWithMinMaxVars:0')
a1_fixed = g.get_tensor_by_name('xw_plus_b_1/act_quant/FakeQuantWithMinMaxVars:0')

xw_plus_b/weights_quant/min/Initializer/Const
xw_plus_b/weights_quant/min
xw_plus_b/weights_quant/min/Assign
xw_plus_b/weights_quant/min/read
xw_plus_b/weights_quant/max/Initializer/Const
xw_plus_b/weights_quant/max
xw_plus_b/weights_quant/max/Assign
xw_plus_b/weights_quant/max/read
xw_plus_b/weights_quant/FakeQuantWithMinMaxVars
xw_plus_b_1/weights_quant/min/Initializer/Const
xw_plus_b_1/weights_quant/min
xw_plus_b_1/weights_quant/min/Assign
xw_plus_b_1/weights_quant/min/read
xw_plus_b_1/weights_quant/max/Initializer/Const
xw_plus_b_1/weights_quant/max
xw_plus_b_1/weights_quant/max/Assign
xw_plus_b_1/weights_quant/max/read
xw_plus_b_1/weights_quant/FakeQuantWithMinMaxVars
xw_plus_b_2/weights_quant/min/Initializer/Const
xw_plus_b_2/weights_quant/min
xw_plus_b_2/weights_quant/min/Assign
xw_plus_b_2/weights_quant/min/read
xw_plus_b_2/weights_quant/max/Initializer/Const
xw_plus_b_2/weights_quant/max
xw_plus_b_2/weights_quant/max/Assign
xw_plus_b_2/weights_quant/max/read
xw_plus_b_2/weight

In [71]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #load pretrained model
    saver.restore(sess, './results/model')

    all_p = sess.run(tf.global_variables())

    #check parameters
    np.set_printoptions(precision=4, suppress = True)
    print('-'*50)
    print('variables')
    for p, pp in zip(all_p, tf.global_variables()):
        if '1' in pp.name:
            print(pp.name)
            if 'min' in pp.name or 'max' in pp.name:
                print(p)
            else:
                print(p[0:5])

    w1f, a1f = sess.run(
            [w1_fixed, a1_fixed],
            feed_dict = {x:dataset.test.images}
            )
    w_clip = (6.0 + 6.0) / (2**_N_W-2) 
    a_clip = (6.0 - 0.0) / (2**_N_A-1) 
    print('w_1')
    print(w1f[0:5])
    print('delta : ', w_clip)
    print('/delta : ', w1f[0:5] / w_clip)
    print('a_1')
    print(a1f[0:5])
    print('delta : ', a_clip)
    print('/delta : ', a1f[0:5] / a_clip)

INFO:tensorflow:Restoring parameters from ./results/model
--------------------------------------------------
variables
w_1:0
[[-0.0332 -0.0602 -0.0491 ...  0.0659 -0.0367  0.0404]
 [-0.0904  0.0497 -0.0422 ...  0.0858 -0.088  -0.0006]
 [ 0.077   0.0667 -0.0383 ...  0.0534 -0.072  -0.0215]
 [-0.063  -0.0265 -0.0032 ... -0.081  -0.0529 -0.0389]
 [ 0.0216  0.067   0.0033 ... -0.0635 -0.0401  0.0316]]
b_1:0
[ 0.0073  0.0068 -0.0006  0.011   0.0257]
xw_plus_b_1/weights_quant/min:0
-6.0
xw_plus_b_1/weights_quant/max:0
6.0
xw_plus_b_1/act_quant/min:0
0.0
xw_plus_b_1/act_quant/max:0
6.0
w_1
[[-0.0472 -0.0472 -0.0472 ...  0.0472 -0.0472  0.0472]
 [-0.0945  0.0472 -0.0472 ...  0.0945 -0.0945 -0.    ]
 [ 0.0945  0.0472 -0.0472 ...  0.0472 -0.0945 -0.    ]
 [-0.0472 -0.0472 -0.     ... -0.0945 -0.0472 -0.0472]
 [-0.      0.0472 -0.     ... -0.0472 -0.0472  0.0472]]
delta :  0.047244094488188976
/delta :  [[-1. -1. -1. ...  1. -1.  1.]
 [-2.  1. -1. ...  2. -2. -0.]
 [ 2.  1. -1. ...  1. -2. -0.]
 